In [1]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.express as px
import csv
import textwrap

import os, shutil
import numpy as np
import csv
import pandas as pd
from os import path
import pickle

import plotly.graph_objects as go

import torch
#from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from transformers import AutoTokenizer, AutoModel

/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
import preprocess_for_context_atlas

/usr/local/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.11/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

device :  cpu


/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
# initialize BERT model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device : ", device)

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
model = model.to(device)



device :  cpu


/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Choose the word we want to visualize

In [4]:
sources = ["coca", "acl"]
word = "model"
k_means_n = 5

### Fetch Data for the word 

In [5]:
dfs = []
for source in sources:
    df = pd.read_csv('./collected_tokens/{}/{}.csv'.format(source,word))
    df['source'] = source

    # get senteces that are less than 150 in length
    df[
        df["sentence"].apply(lambda x: len(x) > 100)
    ]

    # get senteces that are less than 150 in length
    df["sentence"] = df["sentence"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii', 'ignore'))

    ####### REAL ######
    # Take at most 200 sentences.
    sample_df = df.sample(1000, random_state=42) # use a fixed seed for reproducibility
    
    ##### DEBUG########
    # Take at most 2 sentences.
    #sample_df = df.sample(200)
    
    dfs.append(sample_df)
    
    
df = pd.concat(dfs)
df

,Unnamed: 0,corpus_id,sentence,start_idx,end_idx,source
5062,5062,156200,The fit of each of three nonrecursive models (...,24,25,coca
11931,11931,159103,The general linear model applied was : # Due t...,3,4,coca
13938,13938,136866,"Role model o My mom . """,1,2,coca
9932,9932,158417,"Albert , Sci-Fi Club alum from 2006 , felt tha...",29,30,coca
9345,9345,158048,"Chronic exposure to high-intensity light ( 2,0...",49,50,coca
...,...,...,...,...,...,...
131383,131383,18443059,A radically new approach is needed to address ...,24,25,acl
684915,684915,221816590,"In many cases, the model struggles to distingu...",5,6,acl
694830,694830,222133423,A.4.1 Word Embedding Dimension To decide what ...,32,33,acl
372177,372177,15142037,"For language model scoring, we use the SRILM t...",21,22,acl


In [6]:
sample_df = df


In [11]:
filename = 'viz_data/{}.pkl'.format(word)

if not path.exists(filename):
    # get the vectors
    data = preprocess_for_context_atlas.neighbors(word, sample_df)
    # save data
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
else:
    with open(filename, 'rb') as f:
        data = pickle.load(f)

Getting embeddings for 2000 sentences 


/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

Preprocess returns a dict with three array-containing keys. sentence, pos, and data. Later we can add to this preprocessing if we want to add more metadata. For now, we separate them out. would perhaps be better to put them all in a single df and use that to build our plot. TODO if you go down that road but unnecessary now. That would involve making a DF from the labels and one from the data and joining them together.

In [ ]:
sents = [label['sentence'] for label in data['labels'] ]
pos = [label['pos'] for label in data['labels'] ]

these are the layers. We select layer 7 (0 indexed)

In [ ]:
layer = 7


In [ ]:
# get clusters labels for this layer

cluster_ids = data['clusters'][layer]
cluster_ids

In [ ]:

points = data['data'][layer]
points = np.array(points)
umap_x = points[:,0]
umap_y = points[:,1]

In [ ]:
plot_dict = {"umap_x": umap_x,
             "umap_y": umap_y,
             "pos": pos,
             "cluster": cluster_ids,
             "sents": sents,
            "wrapped_sents": list(map((lambda x: x.replace('\n', '<br>')),sents)),
             "source": df['source']
}
df = pd.DataFrame.from_records(plot_dict)
df.head(5)

In [ ]:
#fig.write_html("visualizations/{}_{}_{}.html".format(corpus, word, layer))

Experiment in doing a dropdown

In [ ]:
# How to change plot data using dropdowns
#
# This example shows how to manually add traces
# to the plot and configure the dropdown to only
# show the specific traces you allow.

fig = go.Figure()
#plt.figure(figsize=(16,16))

fig = px.scatter(
                #color = df['pos'],
                 width=1000, height=1000
                )

# """
# create color list based on POS
# """
# #create dict with unique color values
pos = [label['pos'] for label in data['labels'] ]
color_dict={}
x=0
for p in pos:
    if p not in color_dict:
        color_dict[p]=x
        x += 1    
#create color list
color_list=[]
for p in pos:
    color_list.append(color_dict[p])


"""
create color list based on data source
"""
source_dict = {}
for p in sources:
    if p not in source_dict:
        source_dict[p]=x
        x += 1
#create color list
source_list=[]
for p in df['source']:
    source_list.append(source_dict[p])

# create a column of formatted text
df["wrapped_sents"] = df["sents"].str.wrap(50)
df["wrapped_sents"] = df["wrapped_sents"].apply(lambda x: x.replace('\n', '<br>'))

"""
create traces for different layers
"""
for layer in range(12):
    points = data['data'][layer]
    clusters = data['clusters'][layer]
    points = np.array(points)
    fig.add_trace(
        go.Scatter(
            mode='markers',
            marker=dict(
                    size=4,
                    color= source_list,   # pos # cluster_list # use color_list to set color to an array/list of desired values
                    colorscale='Viridis',   # choose a colorscale
                    #opacity=0.8
                ),

            x = points[:,0],
            y = points[:,1],
            hovertext=df["wrapped_sents"],
            
            name = layer
        )
    )

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )


"""
create dropdown menu to show one set of points at a time
"""
    
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),


             
             dict(label = 'Layer 0',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 0',
                           'showlegend':True}]),
             dict(label = 'Layer 1',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 1',
                           'showlegend':True}]),
             dict(label = 'Layer 2',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 2',
                           'showlegend':True}]),
             dict(label = 'Layer 3',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 3',
                           'showlegend':True}]),
             dict(label = 'Layer 4',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 4',
                           'showlegend':True}]),
             dict(label = 'Layer 6',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False, True, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 6',
                           'showlegend':True}]),
             dict(label = 'Layer 7',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 7',
                           'showlegend':True}]),
              dict(label = 'Layer 8',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 8',
                           'showlegend':True}]),
             dict(label = 'Layer 9',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 9',
                           'showlegend':True}]),
             dict(label = 'Layer 10',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 10',
                           'showlegend':True}]),
             dict(label = 'Layer 11',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True]}, # the index of True aligns with the indices of plot traces
                          {'title': 'Layer 11',
                           'showlegend':True}]),

            ])
        )
    ])

fig.update_layout(hoverlabel={
    "align": "right",
    "bgcolor": "white"
})
fig.update_layout(hoverlabel_font={
    "size": 9,
    "color": "black"
})


fig.show()

### 

In [ ]:
fig.write_html("visualizations/{}_{}_full.html".format(corpus, word))